## Libraries

In [43]:
import pandas as pd
import numpy as np
import scipy.stats as stat

from math import sqrt
from mlgear.utils import show, display_columns
from surveyweights import normalize_weights


def margin_of_error(n=None, sd=None, p=None, type='proportion', interval_size=0.95):
    z_lookup = {0.8: 1.28, 0.85: 1.44, 0.9: 1.65, 0.95: 1.96, 0.99: 2.58}
    if interval_size not in z_lookup.keys():
        raise ValueError('{} not a valid `interval_size` - must be {}'.format(interval_size,
                                                                              ', '.join(list(z_lookup.keys()))))
    if type == 'proportion':
        se = sqrt(p * (1 - p)) / sqrt(n)
    elif type == 'continuous':
        se = sd / sqrt(n)
    else:
        raise ValueError('{} not a valid `type` - must be proportion or continuous')
    
    z = z_lookup[interval_size]
    return se * z


def print_pct(pct, digits=0):
    pct = pct * 100
    pct = np.round(pct, digits)
    if pct >= 100:
        if digits == 0:
            val = '>99.0%'
        else:
            val = '>99.'
            for d in range(digits - 1):
                val += '9'
            val += '0%'
    elif pct <= 0:
        if digits == 0:
            val = '<0.1%'
        else:
            val = '<0.'
            for d in range(digits - 1):
                val += '0'
            val += '1%'
    else:
        val = '{}%'.format(pct)
    return val


def calc_result(for_vote, against_vote, n, interval=0.8):
    GENERAL_POLLING_ERROR = 8.0
    TIME_SHIFT_ERROR = 4.0
    N_SIMS = 4000
    
    for_moe = margin_of_error(n=n, p=for_vote/100, interval_size=interval)
    against_moe = margin_of_error(n=n, p=against_vote/100, interval_size=interval)
    undecided = 100 - for_vote - against_vote
    mean = for_vote + undecided * 0.25
    raw_moe = for_moe * 100 + against_moe * 100
    
    allocate_undecided = undecided * 0.4
    margin = raw_moe + allocate_undecided + GENERAL_POLLING_ERROR + TIME_SHIFT_ERROR
    
    cdf_value = 0.5 + 0.5 * interval
    normed_sigma = stat.norm.ppf(cdf_value)
    sigma = margin / 100 / normed_sigma
    
    sims = np.random.normal(mean / 100, sigma, N_SIMS)
    chance_pass = np.sum([s > 0.5 for s in sims]) / N_SIMS
    low, high = np.percentile(sims, [20, 80]) * 100
    
    return {'mean': mean, 'high': high, 'low': low, 'n': n,
            'raw_moe': raw_moe, 'margin': margin, 'sigma': sigma, 'chance_pass': chance_pass}


def print_result(mean, high, low, n, raw_moe, margin, sigma, chance_pass):
    mean = np.round(mean, 1)
    first = np.round(high, 1)
    second = np.round(low, 1)
    sigma = np.round(sigma * 100, 1)
    raw_moe = np.round(raw_moe, 1)
    margin = np.round(margin, 1)
    chance_pass = print_pct(chance_pass, 1)
    if second < first:
        _ = first
        first = second
        second = _
    print(('Result {} (80% CI: {} to {}) (N={}) (raw_moe={}pts, margin={}pts, '
           'sigma={}pts) ({} likely to pass)').format(mean,
                                                      first,
                                                      second,
                                                      n,
                                                      raw_moe,
                                                      margin,
                                                      sigma,
                                                      chance_pass))
    print(('{} (80% CI: {} to {}) ({})').format(mean,
                                                first,
                                                second,
                                                chance_pass))
    print('-')

## Load Processed Data

In [44]:
survey = pd.read_csv('repsonses_processed_national_weighted.csv').fillna('Not presented')
ca_national_survey = pd.read_csv('repsonses_processed_ca_weighted.csv').fillna('Not presented')

/Users/peterhurford/.virtualenvs/dev/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,40,41,42,43,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Analysis

In [45]:
for var in sorted(list(survey.columns)):
    print('## {} ##'.format(var))
    print('unweighted')
    print(survey[var].value_counts(normalize=True)[sorted(survey[var].unique())] * 100)
    print('-')
    print('weighted')
    print(survey[var].value_counts(normalize=True) * survey.groupby(var)['weight'].mean() * 100)
    print('-')
    print('weighted + RV')
    print(survey[var].value_counts(normalize=True) * survey.groupby(var)['rv_weight'].mean() * 100)
    print('-')
    print('weighted + LV')
    print(survey[var].value_counts(normalize=True) * survey.groupby(var)['lv_weight'].mean() * 100)
    print('-')
    print('-')
    print('-')

## age ##
unweighted
18-24    22.775081
25-44    59.425566
45-64    14.927184
65+       2.872168
Name: age, dtype: float64
-
weighted
18-24    13.123520
25-44    35.142462
45-64    34.690959
65+      17.043060
dtype: float64
-
weighted + RV
18-24    11.561746
25-44    35.477146
45-64    35.934752
65+      17.026356
dtype: float64
-
weighted + LV
18-24    11.796761
25-44    33.741110
45-64    36.163509
65+      18.298619
dtype: float64
-
-
-
## agree_11hrs_sleep ##
unweighted
Agree          5.885922
Disagree      90.594660
Don't know     3.519417
Name: agree_11hrs_sleep, dtype: float64
-
weighted
Agree          7.839353
Disagree      89.244575
Don't know     2.916072
dtype: float64
-
weighted + RV
Agree          8.267941
Disagree      89.043663
Don't know     2.688396
dtype: float64
-
weighted + LV
Agree          8.567697
Disagree      88.700752
Don't know     2.731551
dtype: float64
-
-
-
## agree_40hrs_work ##
unweighted
Agree         35.315534
Disagree      53.721683
Don't know    10

-
weighted + LV
Agree            50.685905
Disagree          8.536916
Don't know       28.713125
Not presented    12.064054
dtype: float64
-
-
-
## agree_pain_chickens ##
unweighted
Agree            78.863269
Disagree          1.193366
Don't know        2.588997
Not presented    17.354369
Name: agree_pain_chickens, dtype: float64
-
weighted
Agree            82.026227
Disagree          2.412331
Don't know        3.825579
Not presented    11.735864
dtype: float64
-
weighted + RV
Agree            82.788041
Disagree          2.676931
Don't know        2.569244
Not presented    11.965784
dtype: float64
-
weighted + LV
Agree            82.483094
Disagree          2.682923
Don't know        2.769929
Not presented    12.064054
dtype: float64
-
-
-
## agree_pain_dogs ##
unweighted
Agree            80.481392
Disagree          0.950647
Don't know        1.213592
Not presented    17.354369
Name: agree_pain_dogs, dtype: float64
-
weighted
Agree            85.197215
Disagree          1.707374
Don't 

Agree            35.536651
Disagree         35.152604
Don't know       17.246692
Not presented    12.064054
dtype: float64
-
-
-
## agree_vote_cand_reg_farming ##
unweighted
Agree            48.260518
Disagree          8.151294
Don't know       26.233819
Not presented    17.354369
Name: agree_vote_cand_reg_farming, dtype: float64
-
weighted
Agree            46.164319
Disagree         11.547672
Don't know       30.552145
Not presented    11.735864
dtype: float64
-
weighted + RV
Agree            47.066009
Disagree         12.050256
Don't know       28.917950
Not presented    11.965784
dtype: float64
-
weighted + LV
Agree            46.546200
Disagree         12.078322
Don't know       29.311424
Not presented    12.064054
dtype: float64
-
-
-
## agree_vote_gay ##
unweighted
Agree            15.635113
Disagree         23.220065
Don't know       43.790453
Not presented    17.354369
Name: agree_vote_gay, dtype: float64
-
weighted
Agree            13.863835
Disagree         35.783807
Don't kn

Can trust               33.217156
Can't be too careful    63.161974
Don’t know               3.620871
dtype: float64
-
weighted + LV
Can trust               33.968439
Can't be too careful    62.357357
Don’t know               3.674204
dtype: float64
-
-
-
## income ##
unweighted
Between $100,000 and $150,000    10.659385
Between $15,000 and $49,999      32.686084
Between $50,000 and $74,999      17.698220
Between $75,000 and $99,999      11.690939
Over $150,000                     5.238673
Under $15,000                    22.026699
Name: income, dtype: float64
-
weighted
Between $100,000 and $150,000    14.808665
Between $15,000 and $49,999      29.932401
Between $50,000 and $74,999      17.214574
Between $75,000 and $99,999      12.460789
Over $150,000                    15.300727
Under $15,000                    10.282843
dtype: float64
-
weighted + RV
Between $100,000 and $150,000    15.972102
Between $15,000 and $49,999      27.985987
Between $50,000 and $74,999      17.813290
Betw

False    91.376401
True      8.623599
dtype: float64
-
weighted + RV
False    90.780039
True      9.219961
dtype: float64
-
weighted + LV
False    90.415805
True      9.584195
dtype: float64
-
-
-
## news_cnn ##
unweighted
True    47.390777
Name: news_cnn, dtype: float64
-
weighted
False    53.906965
True     46.093035
dtype: float64
-
weighted + RV
False    51.857498
True     48.142502
dtype: float64
-
weighted + LV
False    52.767259
True     47.232741
dtype: float64
-
-
-
## news_daily_wire ##
unweighted
True    5.78479
Name: news_daily_wire, dtype: float64
-
weighted
False    93.102479
True      6.897521
dtype: float64
-
weighted + RV
False    92.749196
True      7.250804
dtype: float64
-
weighted + LV
False    93.229628
True      6.770372
dtype: float64
-
-
-
## news_fox ##
unweighted
True    24.615696
Name: news_fox, dtype: float64
-
weighted
False    59.271193
True     40.728807
dtype: float64
-
weighted + RV
False    58.181124
True     41.818876
dtype: float64
-
weighted + LV
F

...

dtype: float64
-
-
-
## urban_rural ##
unweighted
Rural       11.974110
Suburban    54.854369
Urban       33.171521
Name: urban_rural, dtype: float64
-
weighted
Rural       14.072862
Suburban    55.105781
Urban       30.821358
dtype: float64
-
weighted + RV
Rural       13.678357
Suburban    55.285064
Urban       31.036579
dtype: float64
-
weighted + LV
Rural       13.860703
Suburban    55.734200
Urban       30.405097
dtype: float64
-
-
-
## vegan ##
unweighted
True    1.132686
Name: vegan, dtype: float64
-
weighted
False    99.313466
True      0.686534
dtype: float64
-
weighted + RV
False    99.492656
True      0.507344
dtype: float64
-
weighted + LV
False    99.469759
True      0.530241
dtype: float64
-
-
-
## vegetarian ##
unweighted
True    4.126214
Name: vegetarian, dtype: float64
-
weighted
False    97.799743
True      2.200257
dtype: float64
-
weighted + RV
False    97.957854
True      2.042146
dtype: float64
-
weighted + LV
False    98.290471
True      1.709529
dtype: float64
-


dtype: float64
-
weighted + LV
Don’t know / Undecided     5.169412
Not presented             67.476838
Vote against / Oppose     12.072060
Vote for / Support        15.281690
dtype: float64
-
-
-
## vote_measure_ca_prop21_rent_control ##
unweighted
Don’t know / Undecided     8.717638
Not presented             65.898058
Vote against / Oppose      8.818770
Vote for / Support        16.565534
Name: vote_measure_ca_prop21_rent_control, dtype: float64
-
weighted
Don’t know / Undecided     8.668416
Not presented             69.696438
Vote against / Oppose      8.928368
Vote for / Support        12.706777
dtype: float64
-
weighted + RV
Don’t know / Undecided     7.610319
Not presented             69.994818
Vote against / Oppose      8.995311
Vote for / Support        13.399551
dtype: float64
-
weighted + LV
Don’t know / Undecided     7.616988
Not presented             69.525098
Vote against / Oppose      9.243917
Vote for / Support        13.613997
dtype: float64
-
-
-
## vote_measure_ca_prop

Don’t know / Undecided     1.270444
Not presented             82.411436
Vote against / Oppose      5.453007
Vote for / Support        10.865113
dtype: float64
-
-
-
## vote_measure_mi_prop2_electronic_privacy ##
unweighted
Don’t know / Undecided     1.031553
Not presented             86.508900
Vote against / Oppose      1.881068
Vote for / Support        10.578479
Name: vote_measure_mi_prop2_electronic_privacy, dtype: float64
-
weighted
Don’t know / Undecided     1.317572
Not presented             85.491526
Vote against / Oppose      2.643380
Vote for / Support        10.547522
dtype: float64
-
weighted + RV
Don’t know / Undecided     1.187212
Not presented             85.503052
Vote against / Oppose      2.334498
Vote for / Support        10.975238
dtype: float64
-
weighted + LV
Don’t know / Undecided     1.259065
Not presented             85.077495
Vote against / Oppose      2.264319
Vote for / Support        11.399120
dtype: float64
-
-
-
## vote_measure_min_wage ##
unweighted
Don’t

Don’t know / Undecided     3.135113
Not presented             85.942557
Vote against / Oppose      5.845469
Vote for / Support         5.076861
Name: vote_measure_utah_hunt, dtype: float64
-
weighted
Don’t know / Undecided     3.108755
Not presented             81.502662
Vote against / Oppose      8.604010
Vote for / Support         6.784573
dtype: float64
-
weighted + RV
Don’t know / Undecided     2.663820
Not presented             82.577887
Vote against / Oppose      7.667331
Vote for / Support         7.090962
dtype: float64
-
weighted + LV
Don’t know / Undecided     2.472170
Not presented             82.243954
Vote against / Oppose      8.039308
Vote for / Support         7.244567
dtype: float64
-
-
-
## vote_method ##
unweighted
By mail          26.779935
Don't know        0.040453
In person         7.746764
Not presented    63.834951
Other             1.597896
Name: vote_method, dtype: float64
-
weighted
By mail          20.507531
Don't know        0.175772
In person        10.90

Another candidate                3.524478
Do not intend to vote            1.343004
Donald Trump, the Republican    40.394193
I did not vote                   0.081609
Joe Biden, the Democrat         50.277412
Not decided                      4.185684
Unsure                           0.193619
dtype: float64
-
weighted + LV
Another candidate                3.526642
Do not intend to vote            1.329739
Donald Trump, the Republican    40.762486
I did not vote                   0.086889
Joe Biden, the Democrat         50.571124
Not decided                      3.516972
Unsure                           0.206147
dtype: float64
-
-
-
## weight ##
unweighted
0.006996     0.020227
0.007054     0.020227
0.007430     0.020227
0.008775     0.020227
0.009202     0.020227
               ...   
36.785517    0.020227
40.631258    0.020227
49.382907    0.020227
53.113744    0.020227
58.113197    0.020227
Name: weight, Length: 4585, dtype: float64
-
weighted
0.006996     0.000142
0.007054     0.000

In [48]:
senate_states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'Colorado', 'Delaware', 'Georgia',
                 'Idaho', 'Illinois', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Massachusetts',
                 'Michigan', 'Minnesota', 'Mississippi', 'Montana', 'Nebraska', 'New Hampshire',
                 'New Jersey', 'New Mexico', 'North Carolina', 'Oklahoma', 'Oregon', 'Rhode Island',
                 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Virginia', 'West Virginia', 'Wyoming']

survey_ = survey.loc[survey['state'].isin(senate_states)]
survey_['weight'] = normalize_weights(survey_['weight'])
survey_['rv_weight'] = normalize_weights(survey_['rv_weight'])
survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])

print('## UNWEIGHTED ##')
n = len(survey_)
options = ['A Democratic candidate', 'A Republican candidate', 'Another candidate', 'Not decided']
votes = survey_['vote_senate'].value_counts(normalize=True) * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
dem = votes['A Democratic candidate']
rep = votes['A Republican candidate']
dem_moe = margin_of_error(n=n, p=dem/100, interval_size=0.8)
rep_moe = margin_of_error(n=n, p=rep/100, interval_size=0.8)
print('-')
print('Dem GS is at +{} (80% CI: +{} to +{})'.format(np.round(dem - rep, 1),
                                                     np.round(dem - rep - dem_moe * 100 - rep_moe * 100, 1),
                                                     np.round(dem - rep + dem_moe * 100 + rep_moe * 100, 1)))

print('-')
print('-')

print('## WEIGHTED ##')
weighted_n = int(np.round(survey_['weight'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_senate'].value_counts(normalize=True) * survey_.groupby('vote_senate')['weight'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
dem = votes['A Democratic candidate']
rep = votes['A Republican candidate']
dem_moe = margin_of_error(n=weighted_n, p=dem/100, interval_size=0.8) + 0.01
rep_moe = margin_of_error(n=weighted_n, p=rep/100, interval_size=0.8) + 0.01
print('-')
print('Dem GS is at +{} (80% CI: +{} to +{})'.format(np.round(dem - rep, 1),
                                                     np.round(dem - rep - dem_moe * 100 - rep_moe * 100, 1),
                                                     np.round(dem - rep + dem_moe * 100 + rep_moe * 100, 1)))
print('-')
print('-')


print('## WEIGHTED + RV ##')
rv_weighted_n = int(np.round(survey_['rv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_senate'].value_counts(normalize=True) * survey_.groupby('vote_senate')['rv_weight'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
dem = votes['A Democratic candidate']
rep = votes['A Republican candidate']
dem_moe = margin_of_error(n=rv_weighted_n, p=dem/100, interval_size=0.8)
rep_moe = margin_of_error(n=rv_weighted_n, p=rep/100, interval_size=0.8)
print('-')
print('Dem GS is at +{} (80% CI: +{} to +{})'.format(np.round(dem - rep, 1),
                                                     np.round(dem - rep - dem_moe * 100 - rep_moe * 100, 1),
                                                     np.round(dem - rep + dem_moe * 100 + rep_moe * 100, 1)))
print('-')
print('-')

print('## WEIGHTED + LV ##')
lv_weighted_n = int(np.round(survey_['lv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_senate'].value_counts(normalize=True) * survey_.groupby('vote_senate')['lv_weight'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
dem = votes['A Democratic candidate']
rep = votes['A Republican candidate']
dem_moe = margin_of_error(n=lv_weighted_n, p=dem/100, interval_size=0.8) + 0.01
rep_moe = margin_of_error(n=lv_weighted_n, p=rep/100, interval_size=0.8) + 0.01
print('-')
print('Dem GS is at +{} (80% CI: +{} to +{})'.format(np.round(dem - rep, 1),
                                                     np.round(dem - rep - dem_moe * 100 - rep_moe * 100, 1),
                                                     np.round(dem - rep + dem_moe * 100 + rep_moe * 100, 1)))

# TODO: Model Trump-Biden, GCB, senate, propositions by weighting, accounting for margin of error, allocating undecideds, accounting for polling error, and accounting for time change

/Users/peterhurford/.virtualenvs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/peterhurford/.virtualenvs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/peterhurford/.virtualenvs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

## UNWEIGHTED ##
A Democratic candidate    64.708861
A Republican candidate    21.518987
Another candidate          1.974684
Not decided               11.797468
Name: vote_senate, dtype: float64
-
Dem GS is at +43.2 (80% CI: +40.8 to +45.6)
-
-
## WEIGHTED ##
A Democratic candidate    45.948725
A Republican candidate    38.505791
Another candidate          1.395151
Not decided               14.150333
dtype: float64
-
Dem GS is at +7.4 (80% CI: +1.3 to +13.6)
-
-
## WEIGHTED + RV ##
A Democratic candidate    48.461194
A Republican candidate    40.832384
Another candidate          1.333900
Not decided                9.372522
dtype: float64
-
Dem GS is at +7.6 (80% CI: +3.4 to +11.9)
-
-
## WEIGHTED + LV ##
A Democratic candidate    48.430670
A Republican candidate    42.350900
Another candidate          1.369429
Not decided                7.849001
dtype: float64
-
Dem GS is at +6.1 (80% CI: +-0.2 to +12.3)


## Ballot Measures

In [49]:
options = ['Vote for / Support', 'Vote against / Oppose', 'Don’t know / Undecided']
measures = [c for c in survey.columns if 'vote_measure' in c and 'meta' not in c]

for measure in measures:
    survey_ = survey.loc[survey[measure].isin(options)].copy()
    survey_['weight'] = normalize_weights(survey_['weight'])
    survey_['rv_weight'] = normalize_weights(survey_['rv_weight'])
    survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])
    
    print('## {} NATIONAL UNWEIGHTED ##'.format(measure))
    n = len(survey_)
    votes = survey_[measure].value_counts(normalize=True) * 100
    votes = votes[options] * (100 / votes[options].sum())
    print(votes)
    print_result(**calc_result(for_vote=votes['Vote for / Support'],
                               against_vote=votes['Vote against / Oppose'],
                               n=n))
    
    print('## {} NATIONAL WEIGHTED ##'.format(measure))
    weighted_n = int(np.round(survey_['weight'].apply(lambda w: 1 if w > 1 else w).sum()))
    votes = survey_[measure].value_counts(normalize=True) * survey_.groupby(measure)['weight'].mean() * 100
    votes = votes[options] * (100 / votes[options].sum())
    print(votes)
    print_result(**calc_result(for_vote=votes['Vote for / Support'],
                               against_vote=votes['Vote against / Oppose'],
                               n=weighted_n))

    print('## {} NATIONAL WEIGHTED + LV ##'.format(measure))
    lv_weighted_n = int(np.round(survey_['lv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
    votes = survey_[measure].value_counts(normalize=True) * survey.groupby(measure)['lv_weight'].mean() * 100
    votes = votes[options] * (100 / votes[options].sum())
    print(votes)
    print_result(**calc_result(for_vote=votes['Vote for / Support'],
                               against_vote=votes['Vote against / Oppose'],
                               n=lv_weighted_n))
    print('-')

## vote_measure_100pct_income_tax NATIONAL UNWEIGHTED ##
Vote for / Support        13.612460
Vote against / Oppose     70.914239
Don’t know / Undecided    15.473301
Name: vote_measure_100pct_income_tax, dtype: float64
Result 17.5 (80% CI: 4.5 to 30.6) (N=4944) (raw_moe=1.5pts, margin=19.6pts, sigma=15.3pts) (2.2% likely to pass)
17.5 (80% CI: 4.5 to 30.6) (2.2%)
-
## vote_measure_100pct_income_tax NATIONAL WEIGHTED ##
Vote for / Support        16.047679
Vote against / Oppose     71.778400
Don’t know / Undecided    12.173921
dtype: float64
Result 19.1 (80% CI: 6.2 to 31.4) (N=2077) (raw_moe=2.3pts, margin=19.2pts, sigma=15.0pts) (2.0% likely to pass)
19.1 (80% CI: 6.2 to 31.4) (2.0%)
-
## vote_measure_100pct_income_tax NATIONAL WEIGHTED + RV ##
Vote for / Support        16.896514
Vote against / Oppose     71.798924
Don’t know / Undecided    11.304563
dtype: float64
Result 19.7 (80% CI: 7.4 to 32.3) (N=1966) (raw_moe=2.4pts, margin=18.9pts, sigma=14.7pts) (1.8% likely to pass)
19.7 (80% 

## vote_measure_ca_prop23_dialysis NATIONAL UNWEIGHTED ##
Vote for / Support        52.472368
Vote against / Oppose     25.421757
Don’t know / Undecided    22.105876
Name: vote_measure_ca_prop23_dialysis, dtype: float64
Result 58.0 (80% CI: 42.1 to 73.6) (N=1719) (raw_moe=2.9pts, margin=23.7pts, sigma=18.5pts) (66.2% likely to pass)
58.0 (80% CI: 42.1 to 73.6) (66.2%)
-
## vote_measure_ca_prop23_dialysis NATIONAL WEIGHTED ##
Vote for / Support        53.493754
Vote against / Oppose     29.181066
Don’t know / Undecided    17.325180
dtype: float64
Result 57.8 (80% CI: 42.1 to 73.6) (N=724) (raw_moe=4.5pts, margin=23.5pts, sigma=18.3pts) (66.3% likely to pass)
57.8 (80% CI: 42.1 to 73.6) (66.3%)
-
## vote_measure_ca_prop23_dialysis NATIONAL WEIGHTED + RV ##
Vote for / Support        56.032954
Vote against / Oppose     27.798778
Don’t know / Undecided    16.168269
dtype: float64
Result 60.1 (80% CI: 45.0 to 74.5) (N=690) (raw_moe=4.6pts, margin=23.1pts, sigma=18.0pts) (71.0% likely to pass

## vote_measure_no_insect_feed NATIONAL UNWEIGHTED ##
Vote for / Support        22.620519
Vote against / Oppose     55.253399
Don’t know / Undecided    22.126082
Name: vote_measure_no_insect_feed, dtype: float64
Result 28.2 (80% CI: 12.5 to 44.7) (N=809) (raw_moe=4.1pts, margin=25.0pts, sigma=19.5pts) (13.0% likely to pass)
28.2 (80% CI: 12.5 to 44.7) (13.0%)
-
## vote_measure_no_insect_feed NATIONAL WEIGHTED ##
Vote for / Support        26.770025
Vote against / Oppose     50.703083
Don’t know / Undecided    22.526892
dtype: float64
Result 32.4 (80% CI: 14.7 to 50.7) (N=335) (raw_moe=6.6pts, margin=27.6pts, sigma=21.5pts) (21.2% likely to pass)
32.4 (80% CI: 14.7 to 50.7) (21.2%)
-
## vote_measure_no_insect_feed NATIONAL WEIGHTED + RV ##
Vote for / Support        27.373257
Vote against / Oppose     51.054377
Don’t know / Undecided    21.572366
dtype: float64
Result 32.8 (80% CI: 14.6 to 51.0) (N=311) (raw_moe=6.9pts, margin=27.5pts, sigma=21.5pts) (20.8% likely to pass)
32.8 (80% CI: 1

Result 66.6 (80% CI: 52.2 to 82.0) (N=301) (raw_moe=6.8pts, margin=22.6pts, sigma=17.6pts) (83.3% likely to pass)
66.6 (80% CI: 52.2 to 82.0) (83.3%)
-
-
## vote_measure_la_a1_abortion NATIONAL UNWEIGHTED ##
Vote for / Support        19.790105
Vote against / Oppose     70.314843
Don’t know / Undecided     9.895052
Name: vote_measure_la_a1_abortion, dtype: float64
Result 22.3 (80% CI: 9.0 to 35.6) (N=667) (raw_moe=4.2pts, margin=20.2pts, sigma=15.8pts) (3.5% likely to pass)
22.3 (80% CI: 9.0 to 35.6) (3.5%)
-
## vote_measure_la_a1_abortion NATIONAL WEIGHTED ##
Vote for / Support        42.807396
Vote against / Oppose     49.157570
Don’t know / Undecided     8.035033
dtype: float64
Result 44.8 (80% CI: 30.1 to 58.8) (N=289) (raw_moe=7.5pts, margin=22.7pts, sigma=17.7pts) (37.9% likely to pass)
44.8 (80% CI: 30.1 to 58.8) (37.9%)
-
## vote_measure_la_a1_abortion NATIONAL WEIGHTED + RV ##
Vote for / Support        43.140871
Vote against / Oppose     50.248882
Don’t know / Undecided     6.6

Vote for / Support        43.754678
Vote against / Oppose     48.112540
Don’t know / Undecided     8.132782
dtype: float64
Result 45.8 (80% CI: 30.5 to 60.6) (N=264) (raw_moe=7.8pts, margin=23.1pts, sigma=18.0pts) (39.8% likely to pass)
45.8 (80% CI: 30.5 to 60.6) (39.8%)
-
## vote_measure_rank_choice NATIONAL WEIGHTED + LV ##
Vote for / Support        42.751239
Vote against / Oppose     48.537370
Don’t know / Undecided     8.711391
dtype: float64
Result 44.9 (80% CI: 29.4 to 60.4) (N=274) (raw_moe=7.7pts, margin=23.2pts, sigma=18.1pts) (38.5% likely to pass)
44.9 (80% CI: 29.4 to 60.4) (38.5%)
-
-
## vote_measure_mi_prop2_electronic_privacy NATIONAL UNWEIGHTED ##
Vote for / Support        78.410795
Vote against / Oppose     13.943028
Don’t know / Undecided     7.646177
Name: vote_measure_mi_prop2_electronic_privacy, dtype: float64
Result 80.3 (80% CI: 67.9 to 92.8) (N=667) (raw_moe=3.8pts, margin=18.8pts, sigma=14.7pts) (98.0% likely to pass)
80.3 (80% CI: 67.9 to 92.8) (98.0%)
-
## v

Vote for / Support        65.317674
Vote against / Oppose     28.856128
Don’t know / Undecided     5.826199
dtype: float64
Result 66.8 (80% CI: 52.7 to 79.6) (N=314) (raw_moe=6.7pts, margin=21.0pts, sigma=16.4pts) (84.2% likely to pass)
66.8 (80% CI: 52.7 to 79.6) (84.2%)
-
## vote_measure_cig_tax NATIONAL WEIGHTED + RV ##
Vote for / Support        65.938597
Vote against / Oppose     28.458448
Don’t know / Undecided     5.602955
dtype: float64
Result 67.3 (80% CI: 54.0 to 81.8) (N=297) (raw_moe=6.9pts, margin=21.1pts, sigma=16.5pts) (86.5% likely to pass)
67.3 (80% CI: 54.0 to 81.8) (86.5%)
-
## vote_measure_cig_tax NATIONAL WEIGHTED + LV ##
Vote for / Support        66.069300
Vote against / Oppose     27.983567
Don’t know / Undecided     5.947133
dtype: float64
Result 67.6 (80% CI: 53.4 to 81.5) (N=300) (raw_moe=6.8pts, margin=21.2pts, sigma=16.5pts) (85.2% likely to pass)
67.6 (80% CI: 53.4 to 81.5) (85.2%)
-
-


In [50]:
for measure in measures:
    if measure != 'vote_measure_100pct_income_tax':
        survey_ = survey.loc[survey[measure].isin(options)].copy()
        survey_['weight'] = normalize_weights(survey_['weight'])
        survey_['rv_weight'] = normalize_weights(survey_['rv_weight'])
        survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])

        print('## {} NATIONAL CORRECTED + WEIGHTED + LV ##'.format(measure))
        lv_weighted_n = int(np.round(survey_['lv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
        votes = survey_[measure].value_counts(normalize=True) * survey_.groupby(measure)['lv_weight'].mean() * 100
        votes = votes[options] * (100 / votes[options].sum())
        print(votes)
        print_result(**calc_result(for_vote=votes['Vote for / Support'],
                                   against_vote=votes['Vote against / Oppose'],
                                   n=lv_weighted_n))
        print('-')

## vote_measure_ca_prop16_affirmative_action NATIONAL CORRECTED ##
Vote for / Support        33.116114
Vote against / Oppose     34.834123
Don’t know / Undecided    32.049763
Name: vote_measure_ca_prop16_affirmative_action, dtype: float64
Result 41.1 (80% CI: 23.7 to 58.9) (N=1688) (raw_moe=3.0pts, margin=27.8pts, sigma=21.7pts) (34.2% likely to pass)
41.1 (80% CI: 23.7 to 58.9) (34.2%)
-
## vote_measure_ca_prop16_affirmative_action NATIONAL CORRECTED + WEIGHTED ##
Vote for / Support        24.455912
Vote against / Oppose     40.847857
Don’t know / Undecided    34.696232
dtype: float64
Result 33.1 (80% CI: 12.6 to 53.2) (N=691) (raw_moe=4.5pts, margin=30.4pts, sigma=23.7pts) (23.6% likely to pass)
33.1 (80% CI: 12.6 to 53.2) (23.6%)
-
## vote_measure_ca_prop16_affirmative_action NATIONAL CORRECTED + WEIGHTED + RV ##
Vote for / Support        24.640907
Vote against / Oppose     43.837760
Don’t know / Undecided    31.521333
dtype: float64
Result 32.5 (80% CI: 12.9 to 50.9) (N=654) (raw_m

Result 53.6 (80% CI: 35.3 to 71.3) (N=683) (raw_moe=4.6pts, margin=27.5pts, sigma=21.5pts) (56.1% likely to pass)
53.6 (80% CI: 35.3 to 71.3) (56.1%)
-
-
## vote_measure_ca_prop17_felon_vote NATIONAL CORRECTED ##
Vote for / Support        60.439560
Vote against / Oppose     16.300366
Don’t know / Undecided    23.260073
Name: vote_measure_ca_prop17_felon_vote, dtype: float64
Result 66.3 (80% CI: 50.8 to 82.0) (N=1638) (raw_moe=2.7pts, margin=24.0pts, sigma=18.7pts) (81.1% likely to pass)
66.3 (80% CI: 50.8 to 82.0) (81.1%)
-
## vote_measure_ca_prop17_felon_vote NATIONAL CORRECTED + WEIGHTED ##
Vote for / Support        49.943008
Vote against / Oppose     24.411421
Don’t know / Undecided    25.645570
dtype: float64
Result 56.4 (80% CI: 38.2 to 74.4) (N=668) (raw_moe=4.6pts, margin=26.9pts, sigma=21.0pts) (61.6% likely to pass)
56.4 (80% CI: 38.2 to 74.4) (61.6%)
-
## vote_measure_ca_prop17_felon_vote NATIONAL CORRECTED + WEIGHTED + RV ##
Vote for / Support        51.54484
Vote against / 

dtype: float64
Result 41.2 (80% CI: 22.9 to 60.7) (N=318) (raw_moe=7.0pts, margin=28.4pts, sigma=22.1pts) (36.3% likely to pass)
41.2 (80% CI: 22.9 to 60.7) (36.3%)
-
## vote_measure_no_labeling NATIONAL CORRECTED + WEIGHTED + LV ##
Vote for / Support        34.689687
Vote against / Oppose     40.863453
Don’t know / Undecided    24.446860
dtype: float64
Result 40.8 (80% CI: 21.6 to 60.0) (N=326) (raw_moe=6.9pts, margin=28.6pts, sigma=22.3pts) (33.8% likely to pass)
40.8 (80% CI: 21.6 to 60.0) (33.8%)
-
-
## vote_measure_cage_free NATIONAL CORRECTED ##
Vote for / Support        68.944844
Vote against / Oppose     11.270983
Don’t know / Undecided    19.784173
Name: vote_measure_cage_free, dtype: float64
Result 73.9 (80% CI: 58.8 to 89.2) (N=834) (raw_moe=3.5pts, margin=23.4pts, sigma=18.2pts) (90.5% likely to pass)
73.9 (80% CI: 58.8 to 89.2) (90.5%)
-
## vote_measure_cage_free NATIONAL CORRECTED + WEIGHTED ##
Vote for / Support        61.702253
Vote against / Oppose     16.020671
Don’t 

dtype: float64
Result 37.6 (80% CI: 14.1 to 59.6) (N=258) (raw_moe=7.4pts, margin=34.4pts, sigma=26.8pts) (31.7% likely to pass)
37.6 (80% CI: 14.1 to 59.6) (31.7%)
-
-
## vote_measure_ne428_payday_cap NATIONAL CORRECTED ##
Vote for / Support        62.390671
Vote against / Oppose     11.078717
Don’t know / Undecided    26.530612
Name: vote_measure_ne428_payday_cap, dtype: float64
Result 69.0 (80% CI: 51.3 to 86.9) (N=686) (raw_moe=3.9pts, margin=26.5pts, sigma=20.7pts) (81.6% likely to pass)
69.0 (80% CI: 51.3 to 86.9) (81.6%)
-
## vote_measure_ne428_payday_cap NATIONAL CORRECTED + WEIGHTED ##
Vote for / Support        59.508028
Vote against / Oppose     10.641450
Don’t know / Undecided    29.850523
dtype: float64
Result 67.0 (80% CI: 47.8 to 85.5) (N=296) (raw_moe=5.9pts, margin=29.9pts, sigma=23.3pts) (77.1% likely to pass)
67.0 (80% CI: 47.8 to 85.5) (77.1%)
-
## vote_measure_ne428_payday_cap NATIONAL CORRECTED + WEIGHTED + RV ##
Vote for / Support        64.364097
Vote against / O

Result 63.3 (80% CI: 46.2 to 80.8) (N=683) (raw_moe=4.3pts, margin=26.4pts, sigma=20.6pts) (74.2% likely to pass)
63.3 (80% CI: 46.2 to 80.8) (74.2%)
-
## vote_measure_family_leave NATIONAL CORRECTED + WEIGHTED ##
Vote for / Support        50.581700
Vote against / Oppose     27.984336
Don’t know / Undecided    21.433964
dtype: float64
Result 55.9 (80% CI: 37.7 to 74.2) (N=281) (raw_moe=7.2pts, margin=27.8pts, sigma=21.7pts) (60.4% likely to pass)
55.9 (80% CI: 37.7 to 74.2) (60.4%)
-
## vote_measure_family_leave NATIONAL CORRECTED + WEIGHTED + RV ##
Vote for / Support        46.192395
Vote against / Oppose     30.492514
Don’t know / Undecided    23.315092
dtype: float64
Result 52.0 (80% CI: 34.1 to 71.4) (N=264) (raw_moe=7.6pts, margin=28.9pts, sigma=22.5pts) (54.6% likely to pass)
52.0 (80% CI: 34.1 to 71.4) (54.6%)
-
## vote_measure_family_leave NATIONAL CORRECTED + WEIGHTED + LV ##
Vote for / Support        46.583277
Vote against / Oppose     30.160507
Don’t know / Undecided    23.2

In [55]:
pd.crosstab(survey['lv_already_voted'], survey['lv_plan_vote'])

lv_plan_vote,Don't know,No,Yes
lv_already_voted,,,
Don't know,8,0,10
No,198,431,2509
Yes,0,1,1787


In [56]:
pd.crosstab(np.round(survey['rv_index'], 0), survey['vote_trump_biden'], normalize='index')

vote_trump_biden,Another candidate,Do not intend to vote,"Donald Trump, the Republican",I did not vote,"Joe Biden, the Democrat",Not decided,Unsure
rv_index,,,,,,,
0,0.020450,0.564417,0.07771,0.000000,0.208589,0.128834,0.000000
1,0.043322,0.014815,0.20651,0.001122,0.689113,0.042424,0.002694


In [57]:
pd.crosstab(np.round(survey['lv_index'], 0), survey['vote_trump_biden'], normalize='index')

vote_trump_biden,Another candidate,Do not intend to vote,"Donald Trump, the Republican",I did not vote,"Joe Biden, the Democrat",Not decided,Unsure
lv_index,,,,,,,
0.0,0.031550,0.463649,0.097394,0.000000,0.24417,0.163237,0.000000
1.0,0.042705,0.000949,0.210439,0.001186,0.71032,0.031554,0.002847


In [58]:
pd.crosstab(survey['vote2016'], survey['vote_trump_biden'], normalize='index')

vote_trump_biden,Another candidate,Do not intend to vote,"Donald Trump, the Republican",I did not vote,"Joe Biden, the Democrat",Not decided,Unsure
vote2016,,,,,,,
Did not vote,0.037897,0.194988,0.093521,0.001222,0.576406,0.092910,0.003056
Donald Trump,0.031317,0.015119,0.777538,0.001080,0.133909,0.037797,0.003240
Hillary Clinton,0.010412,0.002479,0.022806,0.000992,0.945959,0.016857,0.000496
Other,0.249315,0.010959,0.106849,0.000000,0.539726,0.084932,0.008219


In [59]:
pd.crosstab(survey['vote_rep'], survey['vote_senate'], normalize='index')

vote_senate,A Democratic candidate,A Republican candidate,Another candidate,Do not intend to vote,I did not vote,Not decided,There is no Senate election in my state in 2020,There was no Senate election in my state in 2020,Unsure
vote_rep,,,,,,,,,
A Democratic candidate,0.881198,0.012397,0.004821,0.002066,0.001033,0.020661,0.027893,0.045799,0.004132
A Republican candidate,0.051118,0.862620,0.008520,0.003195,0.000000,0.034079,0.027689,0.011715,0.001065
Another candidate,0.173913,0.108696,0.510870,0.010870,0.010870,0.108696,0.021739,0.043478,0.010870
Do not intend to vote,0.015831,0.007916,0.000000,0.839050,0.000000,0.131926,0.005277,0.000000,0.000000
I did not vote,0.150000,0.050000,0.000000,0.000000,0.625000,0.000000,0.000000,0.150000,0.025000
Not decided,0.088632,0.053950,0.007707,0.109827,0.000000,0.709056,0.030829,0.000000,0.000000
Unsure,0.154930,0.070423,0.000000,0.000000,0.014085,0.000000,0.000000,0.084507,0.676056


In [60]:
pd.crosstab(survey['vote_trump_biden'], survey['vote_senate'], normalize='index')

vote_senate,A Democratic candidate,A Republican candidate,Another candidate,Do not intend to vote,I did not vote,Not decided,There is no Senate election in my state in 2020,There was no Senate election in my state in 2020,Unsure
vote_trump_biden,,,,,,,,,
Another candidate,0.315271,0.157635,0.192118,0.019704,0.004926,0.226601,0.034483,0.034483,0.014778
Do not intend to vote,0.014620,0.014620,0.005848,0.926901,0.000000,0.035088,0.002924,0.000000,0.000000
"Donald Trump, the Republican",0.049061,0.784969,0.005219,0.015658,0.005219,0.101253,0.022965,0.012526,0.003132
I did not vote,0.400000,0.000000,0.000000,0.000000,0.400000,0.000000,0.000000,0.200000,0.000000
"Joe Biden, the Democrat",0.801702,0.020177,0.006936,0.011034,0.006936,0.065889,0.028058,0.043506,0.015763
Not decided,0.119048,0.154762,0.019841,0.055556,0.000000,0.619048,0.031746,0.000000,0.000000
Unsure,0.083333,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.583333


In [61]:
pd.crosstab(survey['vote_trump_biden'], survey['vote_rep'], normalize='index')

vote_rep,A Democratic candidate,A Republican candidate,Another candidate,Do not intend to vote,I did not vote,Not decided,Unsure
vote_trump_biden,,,,,,,
Another candidate,0.369458,0.201970,0.182266,0.034483,0.004926,0.177340,0.029557
Do not intend to vote,0.029240,0.011696,0.008772,0.804094,0.000000,0.146199,0.000000
"Donald Trump, the Republican",0.054280,0.802714,0.013570,0.022965,0.005219,0.094990,0.006263
I did not vote,0.400000,0.000000,0.000000,0.000000,0.600000,0.000000,0.000000
"Joe Biden, the Democrat",0.859079,0.025536,0.011034,0.015132,0.009142,0.063682,0.016393
Not decided,0.154762,0.166667,0.015873,0.107143,0.000000,0.555556,0.000000
Unsure,0.083333,0.166667,0.000000,0.000000,0.166667,0.000000,0.583333


In [62]:
pd.crosstab(survey['vote2016'], survey['vote_senate'], normalize='index')

vote_senate,A Democratic candidate,A Republican candidate,Another candidate,Do not intend to vote,I did not vote,Not decided,There is no Senate election in my state in 2020,There was no Senate election in my state in 2020,Unsure
vote2016,,,,,,,,,
Did not vote,0.457213,0.086797,0.008557,0.211491,0.009780,0.174817,0.016504,0.021394,0.013447
Donald Trump,0.119870,0.695464,0.011879,0.024838,0.005400,0.101512,0.022678,0.012959,0.005400
Hillary Clinton,0.821021,0.024789,0.007437,0.004958,0.004462,0.044125,0.032722,0.047595,0.012890
Other,0.484932,0.158904,0.090411,0.016438,0.000000,0.139726,0.035616,0.046575,0.027397


In [63]:
pd.crosstab(survey['vote2016'], survey['vote_rep'], normalize='index')

vote_rep,A Democratic candidate,A Republican candidate,Another candidate,Do not intend to vote,I did not vote,Not decided,Unsure
vote2016,,,,,,,
Did not vote,0.488386,0.092298,0.009780,0.198655,0.012836,0.183374,0.014670
Donald Trump,0.122030,0.718143,0.021598,0.034557,0.006479,0.088553,0.008639
Hillary Clinton,0.889440,0.030739,0.007933,0.006941,0.006445,0.044125,0.014378
Other,0.542466,0.167123,0.109589,0.021918,0.000000,0.131507,0.027397


In [64]:
pd.crosstab(survey['agree_eat_animals_fed_insects'], survey['agree_eat_animals_fed_insects_alt_reverse'])

agree_eat_animals_fed_insects_alt_reverse,Agree,Disagree,Don't know,Not presented
agree_eat_animals_fed_insects,,,,
Agree,246,1275,226,0
Disagree,847,168,118,0
Don't know,246,234,726,0
Not presented,0,0,0,858


In [65]:
pd.crosstab(survey['agree_flies_animal_feed'], survey['agree_flies_animal_feed_alt'])

agree_flies_animal_feed_alt,Agree,Disagree,Don't know,Not presented
agree_flies_animal_feed,,,,
Agree,859,42,89,0
Disagree,183,660,251,0
Don't know,652,230,1120,0
Not presented,0,0,0,858


In [66]:
pd.crosstab(survey['agree_vote_cand_gun_control'], survey['agree_vote_cand_oppose_abortion'])

agree_vote_cand_oppose_abortion,Agree,Disagree,Don't know,Not presented
agree_vote_cand_gun_control,,,,
Agree,443,1684,234,0
Disagree,307,531,168,0
Don't know,142,383,194,0
Not presented,0,0,0,858


In [67]:
pd.crosstab(survey['ffq_none'], survey['vegan'])

vegan,False,True
ffq_none,,
False,4027,0
True,861,56
